![Image](https://substackcdn.com/image/fetch/%24s_%21DJX0%21%2Cf_auto%2Cq_auto%3Agood%2Cfl_progressive%3Asteep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F8cee2652-d401-433c-8b29-0b49c13ce27f_2000x1509.png)

![Image](https://cdn.prod.website-files.com/640f56f76d313bbe39631bfd/650f6755ec1eba8db2438a6b_tool%20use.png)

![Image](https://substackcdn.com/image/fetch/%24s_%21du7b%21%2Cw_1200%2Ch_600%2Cc_fill%2Cf_jpg%2Cq_auto%3Agood%2Cfl_progressive%3Asteep%2Cg_auto/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F06f2cf46-df40-48f9-a798-931222b0f70a_590x592.png)

![Image](https://miro.medium.com/1%2AB-xl5kNgpnLld4_xmqJ1bw.png)

# From Chatbot to **Agentic AI**: A Practical Blueprint for Tool-Enabled LLMs

T

---

## 🧠 1. Architectural Concepts That Make Action Possible

To transform an LLM into a system that **perceives and acts**, we structure the flow around three components:

---

### ⭐ A. **ToolNode** — The Action Executor

A specialized **LangGraph node** that:

* Houses all external tools (search, calculator, stock APIs, etc.)
* Listens for **tool calls** from the LLM
* Executes them, returns structured JSON

Tools are not executed inline — they are orchestrated.

---

### ⚡ B. **ToolsCondition** — Dynamic Route Switch

A **conditional edge** that analyzes LLM intent:

| LLM Output Pattern                | Action             |
| --------------------------------- | ------------------ |
| General/factual question          | ➜ **End response** |
| Task requiring external execution | ➜ **ToolNode**     |

This prevents unnecessary execution and keeps the graph smart.

---

### 🔁 C. **ReAct Loop** (Critical UX Pattern)

Avoid this:

```
ChatNode → ToolNode → End
```

Why? The user sees raw JSON.

Instead use:

```
ChatNode → ToolNode → ChatNode → End
```

This loop lets the LLM interpret tool outputs before responding to the user.

> **Result:** Structured JSON becomes rich language.

---

## 🌀 2. Production Flow (Visual + Sequence)

### 🎯 User Journey

Let’s walk through an example:

**User:** *“What is Tesla’s stock price?”*

**System does:**

1. LLM parses the question
2. LLM signals intention to use stock tool
3. System routes to **ToolNode**
4. Tool fetches structured data
5. Result fed back into LLM
6. LLM crafts user-ready message

---

### 📊 Sequence Diagram

```mermaid
sequenceDiagram
    participant U as User
    participant C1 as ChatNode (LLM)
    participant T as ToolNode
    participant C2 as ChatNode (LLM)

    U->>C1: Ask for stock price
    C1->>T: Execute get_stock_price
    T->>C2: {"ticker":"TSLA","price":323}
    C2->>U: "Tesla is trading at $323."
```

---

## 🛠 3. Backend — Production-Grade Implementation

Here is an implementation using LangGraph and LangChain and ready to deploy.

---

### 🧩 Step 1 — Define and Register Tools

```python
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import tool
import requests

# CALCULATOR TOOL
@tool
def calculator(a: int, b: int, operation: str) -> float:
    """Basic arithmetic operations."""
    ops = {"add": a + b, "subtract": a - b,
           "multiply": a * b, "divide": a / b}
    return ops.get(operation)

# STOCK PRICE TOOL
@tool
def get_stock_price(ticker: str) -> dict:
    """
    Fetches current stock price.
    Replace MOCK with real API in prod.
    """
    # TODO: Replace with AlphaVantage/Polygon call
    return {"ticker": ticker, "price": 323.00}

# WEB SEARCH TOOL
search = DuckDuckGoSearchRun()

# COLLECT ALL TOOLS
tools = [search, calculator, get_stock_price]

# BIND TO LLM
llm_with_tools = llm.bind_tools(tools)
```

---

### 🏗 Step 2 — Build the LangGraph

```python
from langgraph.graph import StateGraph, START, END

def chat_node(state):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

builder = StateGraph(State)

builder.add_node("chatbot", chat_node)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "chatbot")
builder.add_conditional_edges("chatbot", tools_condition)
builder.add_edge("tools", "chatbot")

graph = builder.compile(checkpointer=memory)
```

---

## 🧑‍💻 4. Frontend — Clean UX With Streamlit

**Problem:** Streaming shows raw tool JSON.

**Fix:** Only display messages from the LLM.

```python
from langchain_core.messages import AIMessage

for event in events:
    message = event["messages"][-1]
    if isinstance(message, AIMessage):
        st.write(message.content)
```

**UX Tip:** Use `st.status()` to show live “Running search…” indicators.

---

## 🔍 5. Q&A — Common Pitfalls & Best Practices

| Question                           | Answer                                                  |
| ---------------------------------- | ------------------------------------------------------- |
| **Why the second ChatNode?**       | So the LLM can interpret tool data into natural text.   |
| **Can tools return complex JSON?** | Yes — just ensure the LLM schema understands it.        |
| **How to secure API calls?**       | Use TLS1.3+, API key vaults, rate limits, and retries.  |
| **What if LLM misroutes?**         | Improve tool intent signatures and conditions in Rules. |
| **Local testing?**                 | Use mock tool responses and offline checkpointers.      |

---


